# RFM ANALYSIS CUSTOMER

## IMPORT IMPORTANT MODUL

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from kneed import KneeLocator
from sklearn.cluster import KMeans
import os

## LOAD & PREPROCESS DATA

In [2]:
def load_and_prepare_data(filepath: str) -> pd.DataFrame:
    """
    Load RFM data from a CSV file, clean and preprocess necessary columns.

    Parameters:
    filepath (str): Path to the CSV file.

    Returns:
    pd.DataFrame: Cleaned and preprocessed DataFrame.
    """
    df = pd.read_csv(filepath, delimiter=',')
    df.columns = [col.lower() for col in df.columns]
    
    df['buss_date'] = pd.to_datetime(df['buss_date'])
    df['recency'] = pd.to_datetime(df['recency'])
    df['id'] = pd.Series(range(1, len(df) + 1)).astype(str).str.zfill(7)
    df['kc'] = df['kc'].astype(str).str.zfill(4)
    df['loan_status'] = df['loan_status'].apply(lambda x: 1 if x == 'YES' else 0)
    df.drop(columns=['cif'], inplace=True)
    
    df['recency'] = (df['buss_date'] - df['recency']).dt.days
    return df

In [3]:
# --- Normalization ---
def normalize_columns(df: pd.DataFrame, cols: list) -> (pd.DataFrame, dict):
    """
    Normalize selected columns using min-max scaling.

    Parameters:
    df (pd.DataFrame): DataFrame containing features.
    cols (list): List of column names to normalize.

    Returns:
    pd.DataFrame: DataFrame with normalized columns.
    """
    original_values = {}
    for col in cols:
        min_val, max_val = df[col].min(), df[col].max()
        original_values[col] = (min_val, max_val)
        df[col] = (df[col] - min_val) / (max_val - min_val)
    return df, original_values

In [5]:
# --- Denormalize ---
def denormalize_columns(df: pd.DataFrame, cols: list, original_values: dict) -> pd.DataFrame:
    """     
    Denormalize selected columns using min-max scaling.
    Parameters:
    df (pd.DataFrame): DataFrame containing features.
    cols (list): List of column names to denormalize.

    Returns:
    pd.DataFrame: DataFrame with denormalized columns.
    """ 
    for col in cols:
        min_val, max_val = original_values[col]
        df[col] = df[col] * (max_val - min_val) + min_val
    return df

In [7]:
# --- Elbow Method ---
def find_optimal_k(data: pd.DataFrame, feature: str, max_k: int = 10) -> int:
    """ 
    Determine the optimal number of clusters (k) for K-Means clustering using the Elbow Method.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the feature to be clustered.
    feature (str): The column name (feature) on which K-Means will be applied.
    max_k (int): The maximum number of clusters to test (default is 10).

    Returns:
    int: The optimal number of clusters based on the "elbow" point in the WCSS curve. 
         If the elbow is not detected, defaults to 3.

    Description:
    - Computes Within-Cluster Sum of Squares (WCSS) for k = 1 to max_k.
    - Uses KneeLocator to identify the "elbow" point where adding more clusters yields diminishing returns.
    - Displays a line plot of WCSS vs. number of clusters with the elbow point marked.
    """
    wcss = []
    K = range(1, max_k + 1)
    for k in K:
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
        kmeans.fit(data[[feature]])
        wcss.append(kmeans.inertia_)
    
    kl = KneeLocator(K, wcss, curve="convex", direction="decreasing")
    
    # Plot elbow curve
    plt.figure()
    plt.plot(K, wcss, marker='o')
    if kl.knee:
        plt.axvline(x=kl.knee, color='red', linestyle='--', label=f"Optimal k = {kl.knee}")
    plt.title(f'Elbow Method for {feature.capitalize()}')
    plt.xlabel('Number of Clusters')
    plt.ylabel('WCSS')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    return kl.knee if kl.knee else 3

In [8]:
# --- KMeans Scoring for RFM ---
def assign_rfm_scores(df: pd.DataFrame, n_clusters: int = 5) -> pd.DataFrame:
    """
    Apply KMeans clustering on R, F, M dimensions and assign scores.

    Parameters:
    df (pd.DataFrame): Normalized RFM DataFrame.
    n_clusters (int): Number of clusters to use.

    Returns:
    pd.DataFrame: DataFrame with added r_score, f_score, m_score columns.
    """
    for feature in ['recency', 'frequency', 'monetary']:
        kmeans = KMeans(n_clusters=n_clusters,n_init = 10, random_state=42)
        df[f"{feature[0]}_score"] = kmeans.fit_predict(df[[feature]])
    return df


In [ ]:
# --- Main Execution ---
if __name__ == "__main__":
    filepath = r'C:\Users\LENOVO\Downloads\RFM.txt'
    df = load_and_prepare_data(filepath)

    # Normalize
    df_clean = df.copy()
    df_clean, original_values = normalize_columns(df_clean, ['recency', 'frequency', 'monetary'])

    # Find optimal k
    optimal_ks = {
        feature: find_optimal_k(df_clean, feature)
        for feature in ['recency', 'frequency', 'monetary']
    }

    print("Optimal k per feature:")
    for key, val in optimal_ks.items():
        print(f"{key.capitalize()}: {val}")

    # Assign KMeans scores
    df_clean = assign_rfm_scores(df_clean, n_clusters=5)

    # Map cluster labels (optional: ensure mapping consistency)
    score_mapping = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}
    df_clean[['r_score', 'f_score', 'm_score']] = df_clean[['r_score', 'f_score', 'm_score']].replace(score_mapping)

    # RFM combined score
    df_clean['rfm_score'] = df_clean['r_score'].astype(str) + df_clean['f_score'].astype(str) + df_clean['m_score'].astype(str)

    # Denormalize original RFM columns
    df_clean = denormalize_columns(df_clean, ['recency', 'frequency', 'monetary'], original_values)

    # Merge with RFM segment descriptions
    GSHEET_URL = 'https://docs.google.com/spreadsheets/d/1Aqehai_TsPLH_Lo8vFz80rcZ54OQSq3MTpMKimll6TU/edit#gid=884117664'
    GSHEET_URL_CSV = GSHEET_URL.replace('/edit#gid=', '/export?format=csv&gid=')
    rfm_segments = pd.read_csv(GSHEET_URL_CSV)
    rfm_segments['rfm_score'] = rfm_segments['rfm_score'].astype(str)

    df_clean = df_clean.merge(rfm_segments, how='left', on='rfm_score')

    

In [ ]:
df_clean.head()

In [ ]:
output_path = r"C:\kukuikk\Data Analyst\RFM"
os.makedirs(output_path, exist_ok=True)

kc_target = '0001'

df_kc = df_clean[df_clean['kc'] == kc_target]

if not df_kc.empty:
    df_kc.to_excel(os.path.join(output_path, f"{kc_target}.xlsx"), index=False)
    print(f"Export selesai untuk KC = {kc_target}!")
else:
    print(f"Tidak ada data untuk KC = {kc_target}")